In [2]:
import pandas as pd
import sqlite3 as db
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score,explained_variance_score
from sklearn.cluster import KMeans

Identifying birds that strongly select for one or more habitats

In [3]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [9]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [6]:
#derive habitat cluster labels from FAO values
def kmeans_habitat(cnx: connectDB(),distinctHabitats: int):
    try:
        data = pd.read_sql(sql='SELECT * FROM FAO_by_locId;',con=cnx)
        data = data.drop(columns=['locName']).set_index('locId')
        data.fillna(0,inplace=True)
    #normalize
        maxValue = data.apply(max,axis=1)
        data = data.apply(lambda x: (x/maxValue[x.index]),axis=0)   #min-max normalizing to smooth in proportionality
    #compute kmeans for each locId
        habitat_kmeans = KMeans(n_clusters=distinctHabitats,init='k-means++')
        habitat_kmeans = habitat_kmeans.fit(data.values)
        clusterLabels = habitat_kmeans.labels_
    #define habitats
        habitatFrame = pd.DataFrame(data=clusterLabels,columns=['clusterLabel'],index=data.index).sort_values(by='clusterLabel').reset_index()
        habitatFrame = pd.merge(left=habitatFrame,left_on='locId',right=data,right_on='locId',how='left')
    except Exception as kmeansExc:
        raise kmeansExc
    return habitatFrame     

In [30]:
##TODO #99 replace the update latestUpdate in the function with a trigger in the database
def verify(locId: str,label: int,cnx=connectDB()):
    try:
        cursor = cnx.cursor()
        updateLabel_script = f"UPDATE Hotspots SET habitatLabel = ?, latestUpdate = ? WHERE locId = ?;"
        updatelabel_tuple = (label,str(dt.datetime.today()),locId)
        cursor.execute(updateLabel_script,updatelabel_tuple)
        cnx.commit()
        cursor.close()
    except Exception as ex:
        raise ex
    finally:
        cnx.close()
    return 'pass'

In [31]:
verify(locId='L854460',label=3)

'pass'